In [11]:
import numpy as np
import pandas as pd

pd.set_option('display.max_colwidth', None)

In [12]:
picks = pd.read_parquet('../../Data/2023_2025_pick_trades.parquet')

In [14]:
picks[picks['receiving_owner'] == 'Jose']

,date,receiving_owner,giving_owner,ro_picks_received,ro_picks_gave,go_picks_received,go_picks_gave,season
6,2023-09-06,Jose,Daryl,"[2024_4, 2025_1]",[],[],"[2024_4, 2025_1]",2023
16,2023-09-20,Jose,Bryan,"[2024_2, 2024_4]",[],[],"[2024_2, 2024_4]",2023
20,2023-09-22,Jose,Daryl,[],[2025_1],[2025_1],[],2023
27,2023-10-03,Jose,Luis,[2024_2],[],[],[2024_2],2023
31,2023-10-10,Jose,Luis,[2025_3],[],[],[2025_3],2023
36,2023-10-18,Jose,Weeks,"[2025_2, 2025_2, 2024_2]",[],[],"[2025_2, 2025_2, 2024_2]",2023
46,2023-11-02,Jose,Brigido,"[2024_2, 2025_3]",[],[],"[2024_2, 2025_3]",2023
61,2023-11-21,Jose,Brigido,"[2024_1, 2025_2, 2026_1]",[],[],"[2024_1, 2025_2, 2026_1]",2023
62,2023-11-21,Jose,Luis,[2024_4],[],[],[2024_4],2023
65,2023-11-26,Jose,Bryan,[2025_2],"[2025_2, 2024_4]","[2025_2, 2024_4]",[2025_2],2023


In [7]:
picks

,owner,final_picks,num_picks
0,Jacky,"[2024_4, 2025_4, 2026_2, 2026_4]",5
1,Devonte,"[2024_1, 2024_2, 2025_2, 2025_4, 2026_2, 2026_...",8
2,Daryl,"[2024_1, 2024_2, 2024_4, 2025_1, 2025_1, 2025_...",10
3,Luis,"[2026_1, 2026_2, 2026_3]",6
4,Jose,"[2024_1, 2024_1, 2024_2, 2024_2, 2024_2, 2024_...",27
5,Lalo,"[2024_1, 2024_1, 2024_1, 2024_1, 2024_2, 2024_...",28
6,Marvin,"[2024_1, 2024_3, 2024_3, 2024_4, 2025_1, 2025_...",14
7,Weeks,"[2024_1, 2024_3, 2025_2, 2025_4, 2025_4, 2026_...",11
8,Brigido,"[2024_3, 2025_2, 2025_4, 2026_2, 2026_4, 2026_4]",6
9,Bryan,"[2024_3, 2024_4, 2024_4, 2024_4, 2025_2, 2025_...",16
